In [2]:
!git clone https://github.com/GroupLe/grouple-face-tagger/

Cloning into 'grouple-face-tagger'...
remote: Enumerating objects: 397, done.
remote: Counting objects: 100% (397/397), done.
remote: Compressing objects: 100% (291/291), done.
remote: Total 397 (delta 142), reused 326 (delta 79), pack-reused 0
Receiving objects: 100% (397/397), 13.88 MiB | 26.56 MiB/s, done.
Resolving deltas: 100% (142/142), done.


In [3]:
!cd grouple-face-tagger && touch __init__.py
!mkdir data

In [9]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import os
import matplotlib.pyplot as plt
from pathlib import Path
import torch
import torch.nn as nn
from tqdm import tqdm
from torch.utils.data import Dataset
import torchvision
import wandb
# import sys
# sys.path.insert(0, "grouple-face-tagger/models/face_detection/")
from snn_model.datasets import TripletPathDataset
from snn_model.model import EmbeddingNet, TripletNet
from snn_model.functions import accuracy
from snn_model.transformations import EmptyTransformation
from statistics import mean

In [2]:
root = Path('../../data/face_detection/')

In [3]:
filecount=0
for dirpath, dirs, files in os.walk(root):
    for filename in files:
        filecount+=1
print(filecount)
print(len(os.listdir(root)))

1659
23


In [78]:
import torchvision.transforms as T

composed = torchvision.transforms.Compose([T.ToTensor(),
                                           T.Resize((128, 128)),
                                           T.RandomChoice((T.ColorJitter(0.05, 0.05, 0.05),
                                                          T.RandomRotation(degrees=(0, 30)),
                                                          EmptyTransformation(),
                                                          EmptyTransformation(),
                                                          EmptyTransformation()))])
                                                          

In [ ]:
siamse_dataset = TripletPathDataset(Path(root), transform = composed)

0it [00:00, ?it/s]

In [6]:
print(round(len(siamse_dataset)*0.8))
print(int(len(siamse_dataset)*0.2))

siamse_dataset_train, siamse_dataset_test = torch.utils.data.random_split(siamse_dataset,
                                                                         (round(len(siamse_dataset)*0.8),
                                                                          int(len(siamse_dataset)*0.2)))

from torch.utils.data import DataLoader
batch_size = 100
train_dl = DataLoader(siamse_dataset_train, batch_size = batch_size, shuffle = True)
test_dl = DataLoader(siamse_dataset_test, batch_size = batch_size, shuffle = True)

1327
331


In [64]:
embedding_net = EmbeddingNet()
model = TripletNet(embedding_net)
criterion = nn.TripletMarginLoss(margin=1, p=2)
optimizer = torch.optim.Adagrad(model.parameters())
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')
# wandb.login(host='wandb.ai' relogin=True)

In [65]:
optimizer.state_dict

<bound method Optimizer.state_dict of Adagrad (
Parameter Group 0
    eps: 1e-10
    initial_accumulator_value: 0
    lr: 0.01
    lr_decay: 0
    weight_decay: 0
)>

In [69]:
import wandb
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [70]:
wandb.init(project="triplet_siamse_network", config={
    "learning_rate": 0.01,
    "architecture": "SNN"
})
config = wandb.config

train/accuracy,0.82
train/loss,0.59918
_runtime,135
_timestamp,1627079381
_step,11


train/accuracy,▇▂▃▄▂▁▂▆▁▆▇█
train/loss,▄▄█▃▇▆▄▁▄▁▂▁
_runtime,▁▂▂▃▄▄▅▆▆▇▇█
_timestamp,▁▂▂▃▄▄▅▆▆▇▇█
_step,▁▂▂▃▄▄▅▅▆▇▇█


In [71]:
for epoch in range(10):
    print('\n\nEpoch', epoch)

    print('train')
    
    train_loss_history = []
    train_accuracy_history = []

    test_accuracy_history = []
    test_loss_history = []
    
    for i, (anchor, positive, negative) in (enumerate(train_dl)):
        optimizer.zero_grad()
        
        pred = model.forward(anchor, positive, negative)
        pred_anchor, pred_positive, pred_negative = pred
        loss = criterion(pred_anchor, pred_positive, pred_negative)

        loss.backward() #count gradients
        optimizer.step() #update weights
        scheduler.step(loss)

        cur_accuracy = accuracy(pred_anchor, pred_positive, pred_negative)
        print(f'  [{i}/{len(train_dl)}] acc {cur_accuracy} loss {loss}')
        
        train_accuracy_history.append(cur_accuracy)
        train_loss_history.append(loss)
        wandb.log({"train/accuracy": cur_accuracy, "train/loss": loss})

    print('train accuracy: ', mean(train_accuracy_history))
    print('train loss: ', sum(train_loss_history)/len(train_loss_history))


    print('test')
    with torch.no_grad():
        for i, (anchor, positive, negative) in enumerate(test_dl):
            pred = model.forward(anchor, positive, negative)
            pred_anchor, pred_positive, pred_negative = pred
            loss = criterion(pred_anchor, pred_positive, pred_negative)
            cur_accuracy = accuracy(pred_anchor, pred_positive, pred_negative)
            test_accuracy_history.append(cur_accuracy)
            wandb.log({"test/accuracy":cur_accuracy})
            wandb.log({"test/loss": loss})
            print(f'  [{i}/{len(train_dl)}] acc {cur_accuracy} loss {loss}')
            
    print('test accuracy: ', mean(test_accuracy_history))




Epoch 0
train
  [0/14] acc 0.78 loss 0.7127691507339478
  [1/14] acc 0.8 loss 0.6291276812553406
  [2/14] acc 0.79 loss 0.632846474647522
  [3/14] acc 0.79 loss 0.545107364654541
  [4/14] acc 0.73 loss 0.7576642036437988
  [5/14] acc 0.87 loss 0.4366939067840576
  [6/14] acc 0.77 loss 0.6170132756233215
  [7/14] acc 0.74 loss 0.7776358127593994
  [8/14] acc 0.81 loss 0.5451846122741699
  [9/14] acc 0.8 loss 0.5780346393585205
  [10/14] acc 0.84 loss 0.4974980056285858
  [11/14] acc 0.85 loss 0.37996453046798706
  [12/14] acc 0.79 loss 0.5725834965705872
  [13/14] acc 0.8148148148148148 loss 0.4691707193851471
train accuracy:  0.7982010582010582
train loss:  tensor(0.5822, grad_fn=<DivBackward0>)
test
  [0/14] acc 0.75 loss 0.766713559627533
  [1/14] acc 0.86 loss 0.5451606512069702
  [2/14] acc 0.79 loss 0.5708218216896057
  [3/14] acc 0.8125 loss 0.5801378488540649
test accuracy:  0.803125


Epoch 1
train
  [0/14] acc 0.8 loss 0.5351629257202148
  [1/14] acc 0.82 loss 0.559620797634

KeyboardInterrupt: 